# Notebook to demonstrate interaction between Spark/Python and dashDB

## Credentials for dashDB

Paste the credentials for dashDB from the Data Sources menu on the right in to the cell below.

## First query to dashDB

### Get the table schema

In [1]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

props = {}
props['user'] = credentials_1['bluadmin']
props['password'] = credentials_1['bluadmin']

# fill in table name
table = credentials_1['username'] + "." + "NYPD_COLL"

data_df = sqlContext.read.jdbc(credentials_1['jdbcurl'],table,properties=props)
data_df.printSchema()

NameError: name 'sc' is not defined

### Query the table and return the 5 rows 

In [ ]:
data_df.take(5)

## Visualize the data

### Load libraries for visualization

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

ImportError: No module named matplotlib

### Convert the data to pandas structure for viusalization

In [ ]:
collisions_pd = data_df[data_df['LATITUDE'] != 0][['LATITUDE', 'LONGITUDE', 'DATE', 'TIME',
                                                               'BOROUGH', 'ON_STREET_NAME', 'CROSS_STREET_NAME',
                                                               'PERSONS_INJURED', 'PERSONS_KILLED',
                                                               'CONTRIBUTING_FACTOR_1']].toPandas()

collisions_pd.columns = ['Latitude', 'Longitude', 'Date', 'Time', 'Borough', 'On Street',
                         'Cross Street', 'Persons Injured', 'Persons Killed', 'Contributing Factor']

### Visualize data

In [ ]:
#adjust settings
plt.figure(figsize=(15,10))

#create scatterplots
plt.scatter(collisions_pd.Longitude, collisions_pd.Latitude, alpha=0.05, s=4, color='darkseagreen')

#adjust more settings
plt.title('Motor Vehicle Collisions in New York City', size=25)
plt.xlim((-74.26,-73.7))
plt.ylim((40.5,40.92))
plt.xlabel('Longitude',size=20)
plt.ylabel('Latitude',size=20)

plt.show()

## Query the k-means cluster data

### Join the base table with the cluster information (in dashDB)

In [ ]:
# fill in table name
table = credentials_1['username'] + "." + "NYPD_COLL_KM"

km_df = sqlContext.read.jdbc(credentials_1['jdbcurl'],table,properties=props)
km_df.printSchema()

km_pd = km_df[km_df['LATITUDE'] != 0][['LATITUDE', 'LONGITUDE', 'DATE', 'TIME',
                                                               'BOROUGH', 'ON_STREET_NAME', 'CROSS_STREET_NAME',
                                                               'PERSONS_INJURED', 'PERSONS_KILLED',
                                                               'CONTRIBUTING_FACTOR_1', 'CLUSTER_ID']].toPandas()

km_pd.columns = ['Latitude', 'Longitude', 'Date', 'Time', 'Borough', 'On Street',
                         'Cross Street', 'Persons Injured', 'Persons Killed', 'Contributing Factor', 'Cluster']


### Visualize data from a single cluster

In [ ]:
clusterid = 2

cl_pd = km_pd[km_pd['Cluster']==clusterid]

plt.figure(figsize=(15,10), dpi=0.1)

plt.scatter(cl_pd.Longitude, cl_pd.Latitude, color='red', s=1, marker ='.')

#adjust more settings
plt.title('Motor Vehicle Collisions in New York City in cluster #'+str(clusterid), size=20)
plt.xlim((-74.26,-73.7))
plt.ylim((40.5,40.92))
plt.xlabel('Longitude',size=20)
plt.ylabel('Latitude',size=20)
plt.show()

## Aggregate data

### Aggregate in Spark

In [ ]:
cluster = km_df.groupBy('CLUSTER_ID').count().sort('count').toPandas()
colors = ['g','0.75','y','k','b','r']
plt.barh(range(4),cluster.sort_values('count', ascending=True)['count'], color=colors)
plt.xlabel('Collisions')
plt.ylabel('Cluster')
plt.title('Total Number of Collisions by Cluster', size=15)
plt.yticks(range(4), cluster['CLUSTER_ID'])
plt.show()

### Install ibmdbpy library

In [ ]:
!pip install ibmdbpy --user --upgrade
!wget -O $HOME/.local/lib/python2.7/site-packages/ibmdbpy/db2jcc4.jar https://ibm.box.com/shared/static/o9qfkdaoio8hj4c1fs37sz8zhjp77rl2.x

### Connect to dashDB using ibmdbpy

In [ ]:
from ibmdbpy import IdaDataBase, IdaDataFrame

username = credentials_1['username']
password = credentials_1['password']

jdbc = credentials_1['jdbcurl'] + ":user=" + username + ";password=" + password
print(jdbc)

idadb = IdaDataBase(jdbc)

### Aggregate data directly in dashDB and visualize results

In [ ]:
cluster = idadb.ida_query("SELECT cluster_id, count(*) as count FROM NYPD_COLL_KM group by cluster_id")

colors = ['g','0.75','y','k','b','r']
plt.barh(range(4),cluster.sort_values('COUNT', ascending=True)['COUNT'], color=colors)
plt.xlabel('Collisions')
plt.ylabel('Cluster')
plt.title('Total Number of Collisions by Cluster', size=15)
plt.yticks(range(4), cluster['CLUSTER_ID'])
plt.show()

## Geospatial analytics

### Distance

Select collisions that are in a range of 100 meters around Times Square (40.758909, -73.985116) and visualize result.

In [ ]:
query = "select latitude, longitude from nypd_coll where location is not null and " \
        "db2gse.ST_Distance(db2gse.ST_Point(latitude, longitude,1), db2gse.ST_Point(40.758909, -73.985116, 1), 'METRE') < 100"
timessquare = idadb.ida_query(query)

plt.figure(figsize=(15,10), dpi=0.1)

plt.scatter(collisions_pd.Longitude, collisions_pd.Latitude, alpha=0.05, s=4, color='darkseagreen')
plt.scatter(timessquare.LONGITUDE, timessquare.LATITUDE, color='red', s=1, marker ='.')

#adjust more settings
plt.title('Motor Vehicle Collisions close to Times Square in New York City ')
plt.xlim((-74.26,-73.7))
plt.ylim((40.5,40.92))
plt.xlabel('Longitude',size=20)
plt.ylabel('Latitude',size=20)
plt.show()

### Polygon

Select Collisions that are around Central Park and visualize result.

In [ ]:
query = "select latitude, longitude from nypd_coll where location is not null and " \
        "db2gse.ST_Intersects(db2gse.ST_Point(latitude, longitude,1), " \
        "db2gse.ST_Polygon('polygon ((40.767804 -73.982646, 40.801022 -73.958432, 40.797178 -73.948613, 40.764000 -73.972981, 40.767804 -73.982646))',1)) = 1"
centralpark = idadb.ida_query(query)

plt.figure(figsize=(15,10), dpi=0.1)

plt.scatter(collisions_pd.Longitude, collisions_pd.Latitude, alpha=0.05, s=4, color='darkseagreen')
plt.scatter(centralpark.LONGITUDE, centralpark.LATITUDE, color='red', s=1, marker ='.')

#adjust more settings
plt.title('Motor Vehicle Collisions close to Central Park in New York City ')
plt.xlim((-74.26,-73.7))
plt.ylim((40.5,40.92))
plt.xlabel('Longitude',size=20)
plt.ylabel('Latitude',size=20)
plt.show()